In [1]:
import utils.utils as utils
import models.xgboost_model as xgb_model
import pandas as pd
import numpy as np

In [7]:
def get_pitch_data():
    # Read csv files of saved pitch data from the MLB 2016-2019 seasons
    pitch_data = pd.read_csv('raw_pitch_data_all_base_v2.csv', index_col=0)
    print("pitch data loaded")
    return pitch_data

def filter_pitch_data(pitch_data):
    pre_filter_rows = len(pitch_data.index)
    pitch_data = pitch_data[pd.notnull(pitch_data['p1_pitch_type'])]
    post_filter_rows = len(pitch_data.index)

    filter_diff = pre_filter_rows - post_filter_rows
    filter_pcnt = (filter_diff)/pre_filter_rows

    print('Removed Null/NaN labeled pitch types rows, filtered %d of %d rows at %f%%' % (filter_diff, pre_filter_rows, filter_pcnt))
    return pitch_data

def drop_pitch_types(pitch_data):
    #
    # Drop rows with unwanted pitchtypes (including automatic ball/strikes, pitchouts, etc)
    #
    pre_filter_rows = len(pitch_data.index)
    pitch_data = utils.drop_unwanted_pitches(pitch_data)
    post_filter_rows = len(pitch_data.index)

    filter_diff = pre_filter_rows - post_filter_rows
    filter_pcnt = (filter_diff)/pre_filter_rows

    print('Removed rows w/ unwanted pitch types, filtered %d of %d rows at %f%%' % (filter_diff, pre_filter_rows, filter_pcnt))
    return pitch_data

def drop_columns(pitch_data):
    #
    # Drop unwanted dataset columns 
    # 

    # ID columns to drop
    id_cols_to_drop=['p1_pitch_id','p0_pitch_id','pitch_data_id','team_id','game_id',
                    'inning_id','half_inning_id','at_bat_id','gid','b1_id','b1_team_id',
                    'team_abbrev']
    pitch_data = utils.drop_columns_by_list(pitch_data,id_cols_to_drop)
    # Pitch data columns to drop
    pitch_cols_to_drop = ['p0_pitch_seqno', 'p1_pitch_seqno', 'p0_inning', 'result_type',
                          'type_confidence', 'p0_at_bat_o', 'p0_pitch_des', 'nasty']
    pitch_data = utils.drop_columns_by_list(pitch_data, pitch_cols_to_drop)

    # Optional pitchf/x data columns to drop
    #pitchfx_cols_to_drop = ['pitch_count_atbat', 'pitch_count_team', 'start_speed', 'spin_dir',
    #                        'x', 'y', 'sz_top', 'sz_bot', 'pfx_x', 'pfx_z', 'px', 'pz',
    #                        'x0', 'y0', 'z0', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'break_y']
    #pitch_data = utils.drop_columns_by_list(pitch_data, pitchfx_cols_to_drop)

    print("dropped cols")
    return pitch_data

def add_run_diff(pitch_data):
    #
    # Create new column of run differential
    #
    pitch_data['run_diff'] = pitch_data['runs_pitcher_team'] - pitch_data['runs_batter_team']
    cols_to_drop=['runs_pitcher_team','runs_batter_team']
    pitch_data = utils.drop_columns_by_list(pitch_data, cols_to_drop)
    print("added run diff")
    return pitch_data

def add_crunch_time(pitch_data):
    #
    # Create new column for crunch time (after 7th inning)
    #
    pitch_data['inning'] = pitch_data['inning'].astype(dtype='int64')
    pitch_data['inning'] = pitch_data['inning'].fillna(0)  # '0' is for unknown inning (Other values are 1-9)
    pitch_data['crunch_time'] = np.where(pitch_data['inning'] > 7, 1, 0)
    cols_to_drop=['inning']
    pitch_data = utils.drop_columns_by_list(pitch_data, cols_to_drop)
    print("added crunch time")
    return pitch_data

def replace_nans(pitch_data):
    #
    # Replace Nulls/NaN values that are left in the remaining object columns
    #
    #
    # Replace Nulls/NaN values that are left in the remaining object columns
    #
    pitch_data['p0_pitch_type'] = pitch_data['p0_pitch_type'].fillna('NP')  # 'NP' is for No Pitch

    pitch_data['result_type_simple'] = pitch_data['result_type_simple'].fillna('X')  # 'X' is for in play

    pitch_data['b1_game_position'] = pitch_data['b1_game_position'].fillna('Unknown')

    pitch_data['b1_bats'] = pitch_data['b1_bats'].fillna('R')  # 'R' is for right handed (Other values are L or S)

    pitch_data['throws'] = pitch_data['throws'].fillna('R')  # 'R' is for right handed (Other value is L)

    #pitch_data['inning'] = pitch_data['inning'].fillna('0')  # '0' is for unknown inning (Other values are 1-9)

    print('Current number of dataframe Null/NaN values: %d' % (pitch_data.isnull().sum().sum()))
    #
    # Fill the rest of Null/NaN values with zero in numeric columns
    #
    replace_dict = {'nasty': 0, 'x': 0, 'y': 0, 'sz_top': 0, 'sz_bot': 0, 'pfx_x': 0, 'pfx_z': 0,
                    'px': 0, 'pz': 0, 'x0': 0, 'y0': 0, 'z0': 0, 'vx0': 0, 'vy0': 0, 'vz0': 0,
                    'ax': 0, 'ay': 0, 'az': 0, 'break_y': 0, 'break_angle': 0, 'break_length': 0,
                    'start_speed': 0, 'end_speed': 0, 'zone': 0, 'outcome': 0, 'spin_rate': 0,
                    'spin_dir': 0, 'pitch_count_at_bat': 0, 'pitch_count_team': 0,
                    'wins': 0, 'losses': 0, 'b1_bat_order': 0}
    pitch_data = pitch_data.fillna(value=replace_dict)

    print('Current number of dataframe Null/NaN values: %d' % (pitch_data.isnull().sum().sum()))

    return pitch_data

def encode_object_data(pitch_data):
    print('Encoding pitch dataframe of shape {}...'.format(pitch_data.shape))

    # Split label column from rest of pitch dataframe then encode
    Y_all = pitch_data.loc[:, 'p1_pitch_type'].copy()
    Y_all = utils.encode_simple_pitch_types(Y_all)

    # Drop label colum from pitch dataframe, then one-hot-encode object columns
    pitch_data = pitch_data.drop('p1_pitch_type', axis=1)
    pitch_data = utils.one_hot_encode(pitch_data,False)

    # Insert label data back into pitch dataframe
    pitch_data['p1_pitch_type'] = Y_all.copy()

    print('Pitch dataframe encoding complete. New shape: {}'.format(pitch_data.shape))
    return pitch_data

def split_train_test(pitch_data):
    pd_train = pitch_data[pitch_data['season']!=2019].copy()
    pd_test = pitch_data[pitch_data['season']==2019].copy()

    print('Shape of ALL training data set is {}'.format(pd_train.shape))
    print('Shape of ALL test data set is {}'.format(pd_test.shape))

    return pd_train,pd_test

def get_pitcher_data(pd_train,pd_test,pitcher_id):
    pd_train_pitcher = pd_train[pd_train['pitcher_id']==pitcher_id].copy()
    pd_test_pitcher = pd_test[pd_test['pitcher_id']==pitcher_id].copy()
    return pd_train_pitcher,pd_test_pitcher


def drop_season_pitch_id_cols(pd_train,pd_test):
    cols_to_drop=['season','pitcher_id']
    pd_test = utils.drop_columns_by_list(pd_test, cols_to_drop)
    pd_train = utils.drop_columns_by_list(pd_train, cols_to_drop)
    return pd_train,pd_test

def get_X_Y(pitch_data,num_pitch_types):
    X = pitch_data.drop('p1_pitch_type',axis=1).copy()
    Y = pitch_data.loc[:,'p1_pitch_type'].copy()
    return X,Y

# Part 1: Get the data and do all necessary feature engineering

In [8]:
pitch_data = get_pitch_data()
pitch_data = filter_pitch_data(pitch_data)
pitch_data = drop_pitch_types(pitch_data)
pitch_data = drop_columns(pitch_data)
pitch_data = add_run_diff(pitch_data)
pitch_data = add_crunch_time(pitch_data)
# Set intended data types of the remaining columns
pitch_data = utils.set_dtypes(pitch_data)
pitch_data['season'] = pitch_data['season'].astype(dtype='int64')
pitch_data['pitcher_id'] = pitch_data['pitcher_id'].astype(dtype='int64')
pitch_data = replace_nans(pitch_data)
pitch_data = encode_object_data(pitch_data)
pd_train,pd_test = split_train_test(pitch_data)
# get the data for top 3 pitchers
pd_train_verlander,pd_test_verlander = get_pitcher_data(pd_train,pd_test,434378)
pd_train_scherzer,pd_test_scherzer= get_pitcher_data(pd_train,pd_test,453286)
pd_train_porcello,pd_test_porcello= get_pitcher_data(pd_train,pd_test,519144)
print('Verlander pitch data rows: train=%d, test=%d.' % (len(pd_train_verlander.index), len(pd_test_verlander.index)))
print('Scherzer pitch data rows: train=%d, test=%d.' % (len(pd_train_scherzer.index), len(pd_test_scherzer.index)))
print('Porcello pitch data rows: train=%d, test=%d.' % (len(pd_train_porcello.index), len(pd_test_porcello.index)))

# Lastly drop season and pitch_id columns
pd_train,pd_test = drop_season_pitch_id_cols(pd_train,pd_test)
pd_train_verlander,pd_test_verlander = drop_season_pitch_id_cols(pd_train_verlander,pd_test_verlander)
pd_train_scherzer,pd_test_scherzer = drop_season_pitch_id_cols(pd_train_scherzer,pd_test_scherzer)
pd_train_porcello,pd_test_porcello = drop_season_pitch_id_cols(pd_train_porcello,pd_test_porcello)

num_pitch_types = 16
# get the NN data for Verlander
X_test_verlander,Y_test_verlander = get_X_Y(pd_test_verlander,num_pitch_types)
X_train_verlander,Y_train_verlander = get_X_Y(pd_train_verlander,num_pitch_types)
num_cols = len(X_test_verlander.iloc[0,:])



pitch data loaded
Removed Null/NaN labeled pitch types rows, filtered 33043 of 2906621 rows at 0.011368%
Removed rows w/ unwanted pitch types, filtered 5700 of 2873578 rows at 0.001984%
dropped cols
added run diff
added crunch time
Current number of dataframe Null/NaN values: 4550085
Current number of dataframe Null/NaN values: 0
Encoding pitch dataframe of shape (2867878, 53)...
Pitch dataframe encoding complete. New shape: (2867878, 83)
Shape of ALL training data set is (2125821, 83)
Shape of ALL test data set is (742057, 83)
Verlander pitch data rows: train=10613, test=3256.
Scherzer pitch data rows: train=10105, test=2657.
Porcello pitch data rows: train=9762, test=2960.


# Part 2: Train an XGBoost Classfier

In [4]:
# get the model the model for verlander
model_verlander = xgb_model.get_multi_class_classifier_model()

#### Step 1: Fix learning rate and number of estimators for tuning tree-based parameters.
In order to decide on boosting parameters, we need to set some initial values of other parameters. Lets take the following values:

* max_depth = 5 : This should be between 3-10. Ive started with 5 but you can choose a different number as well. 4-6 can be good starting points.
* min_child_weight = 1 : A smaller value is chosen because it is a highly imbalanced class problem and leaf nodes can have smaller size groups.
* gamma = 0 : A smaller value like 0.1-0.2 can also be chosen for starting. This will anyways be tuned later.
* subsample, colsample_bytree = 0.8 : This is a commonly used used start value. Typical values range between 0.5-0.9.
* scale_pos_weight- I'll ignore it for now, but if we need to, we'll adjust the weigths by the occurence of each class

Please note that all the above are just initial estimates and will be tuned later. Lets take the default learning rate of 0.1 here and check the optimum number of trees using cv function of xgboost

In [15]:
xgb_model.fit_multi_class_model(model=model_verlander,x_train=X_train_verlander,y_train=Y_train_verlander,x_test=X_test_verlander,y_test=Y_test_verlander,save_location='verlander.bin',useTrainCV=True)

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



Model Report
Accuracy (Train) : 0.7000848016583435
Accuracy (Test) : 0.5276412776412777


In [16]:
n_estimators= model_verlander.n_estimators
print("The optimum number of estimators is {}".format(n_estimators))

The optimum number of estimators is 109


#### Step 2: Tune max_depth and min_child_weight
We tune these first as they will have the highest impact on model outcome. To start with, lets set wider ranges and then we will perform another iteration for smaller ranges.

In [20]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

In [44]:
from sklearn.model_selection import RandomizedSearchCV
gsearch1 = RandomizedSearchCV(
    estimator = xgb_model.get_multi_class_classifier_model(n_estimators=109),
    param_distributions = param_test1,
    scoring='neg_log_loss',
    n_jobs=4,iid=False, 
    cv=5)
gsearch1.fit(X_train_verlander,Y_train_verlander)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([12.15200033, 29.09850039, 19.46975861, 11.35141468, 18.35585723,
         20.09728403, 36.42628832, 39.03776622, 29.13407955, 32.86936731]),
  'std_fit_time': array([1.23697548, 2.80626469, 1.48634652, 1.36350477, 1.54333778,
         0.86944595, 2.82808895, 1.77809605, 1.96830722, 4.70982964]),
  'mean_score_time': array([0.03584948, 0.06521745, 0.05176139, 0.03277383, 0.04855232,
         0.05988235, 0.07033405, 0.09238815, 0.07239819, 0.06583562]),
  'std_score_time': array([0.00606894, 0.00839752, 0.00816775, 0.00333232, 0.00398273,
         0.01022954, 0.00514603, 0.00512027, 0.00761818, 0.01432484]),
  'param_min_child_weight': masked_array(data=[1, 1, 3, 5, 5, 1, 5, 1, 3, 3],
               mask=[False, False, False, False, False, False, False, False,
                     False, False],
         fill_value='?',
              dtype=object),
  'param_max_depth': masked_array(data=[3, 7, 5, 3, 5, 5, 9, 9, 7, 9],
               mask=[False, False, False, Fa

Ideal values for max_depth is 3, and min_child_weight is 1. Lets go one step deeper and look for optimum values. We'll search for values 1 above and below the optimum values because we took an interval of two.

In [48]:
param_test2 = {
 'max_depth':[2,3,4],
 'min_child_weight':[0,1,2]
}

In [49]:
from sklearn.model_selection import RandomizedSearchCV
gsearch2 = RandomizedSearchCV(
    estimator = xgb_model.get_multi_class_classifier_model(n_estimators=109),
    param_distributions = param_test2,
    scoring='neg_log_loss',
    n_jobs=4,iid=False, 
    cv=5)
gsearch2.fit(X_train_verlander,Y_train_verlander)
gsearch2.cv_results_, gsearch1.best_params_, gsearch1.best_score_

/home/paperspace/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


({'mean_fit_time': array([10.04543266,  9.39223962,  9.19669223, 12.85699019, 12.8296618 ,
         11.69350834, 15.60631852, 18.20510759, 13.05649967]),
  'std_fit_time': array([0.70171106, 0.95792164, 1.32077221, 1.15069112, 1.15264124,
         1.11582071, 1.71308089, 1.94518833, 4.11474651]),
  'mean_score_time': array([0.03693185, 0.03107929, 0.02374892, 0.03958178, 0.03579798,
         0.04078298, 0.04433303, 0.0469347 , 0.0386065 ]),
  'std_score_time': array([0.01130107, 0.00793978, 0.00579565, 0.01585669, 0.00808217,
         0.00452127, 0.00973598, 0.00655385, 0.01057118]),
  'param_min_child_weight': masked_array(data=[0, 1, 2, 0, 1, 2, 0, 1, 2],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_max_depth': masked_array(data=[2, 2, 2, 3, 3, 3, 4, 4, 4],
               mask=[False, False, False, False, False, False, False, False,
                     False],


Now I see the optimal max_depth=1, and min_child_weight=3
#### Step 3: Tune gamma
Now lets tune gamma value using the parameters already tuned above. Gamma can take various values but Ill check for 5 values here. You can go into more precise values as.


In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

from sklearn.model_selection import RandomizedSearchCV
gsearch3 = RandomizedSearchCV(
    estimator = xgb_model.get_multi_class_classifier_model(n_estimators=109),
    param_distributions = param_test3,
    scoring='neg_log_loss',
    n_jobs=4,iid=False, 
    cv=5)
gsearch3.fit(X_train_verlander,Y_train_verlander)
gsearch3.cv_results_, gsearch1.best_params_, gsearch1.best_score_

/home/paperspace/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


a gamma value of 0 had the best result. Before proceeding, a good idea would be to re-calibrate the number of boosting rounds for the updated parameters.

In [36]:
model_verlander=utils.xgb_model.get_multi_class_classifier_model()

'x_train_verlander.pd'

In [37]:
utils.save_dataframe(Y_train_verlander,'y_train_verlander.pd')

/home/paperspace/Project/scalable_ml_project/utils/utils.py:261: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  df.to_csv(filename)


'y_train_verlander.pd'

In [38]:
Y_train_verlander.to_csv('y_train_verlander.csv')

/home/paperspace/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [40]:
type(Y_train_verlander)

pandas.core.series.Series

In [42]:
X_train_verlander

,era,wins,losses,b1_stand,b1_height,b1_avg,b1_hr,b1_rbi,b1_bat_order,x,...,p0_pitch_type_FF,p0_pitch_type_FS,p0_pitch_type_FT,p0_pitch_type_KC,p0_pitch_type_KN,p0_pitch_type_NP,p0_pitch_type_SC,p0_pitch_type_SI,p0_pitch_type_SL,p0_pitch_type_UN
11482,4.5,0.0,0.0,0,71,0.000,0,0,1.0,0.00,...,0,0,0,0,0,1,0,0,0,0
11483,4.5,0.0,0.0,0,71,0.000,0,0,1.0,132.97,...,1,0,0,0,0,0,0,0,0,0
11484,4.5,0.0,0.0,1,73,0.000,0,0,2.0,123.98,...,1,0,0,0,0,0,0,0,0,0
11485,4.5,0.0,0.0,1,73,0.000,0,0,2.0,82.20,...,1,0,0,0,0,0,0,0,0,0
11486,4.5,0.0,0.0,1,73,0.000,0,0,2.0,88.14,...,1,0,0,0,0,0,0,0,0,0
11487,4.5,0.0,0.0,1,73,0.000,0,0,2.0,78.50,...,0,0,0,0,0,0,0,0,1,0
11488,4.5,0.0,0.0,1,73,0.000,0,0,2.0,105.11,...,1,0,0,0,0,0,0,0,0,0
11489,4.5,0.0,0.0,1,73,0.000,0,0,2.0,86.47,...,1,0,0,0,0,0,0,0,0,0
11490,4.5,0.0,0.0,0,75,0.000,0,0,3.0,81.40,...,1,0,0,0,0,0,0,0,0,0
11491,4.5,0.0,0.0,1,78,0.000,0,0,4.0,145.59,...,1,0,0,0,0,0,0,0,0,0
